<a href="https://colab.research.google.com/github/hsswkwk/turbo-chainsaw/blob/feature-add-anomaly-detection/notebooks/anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 異常検知（Anomaly detection）
標準的な状態や想定から逸脱しているデータや事象を特定する技術


In [ ]:
import numpy as np
import pandas as pd

## 正規分布に従うデータの異常検知
### 例
製品の温度をセンサーで監視している場合の異常検知
<br>
<br>
### 手法

#### 3$\sigma$法
データが正規分布に従うと仮定し、平均値から標準偏差の3倍以上離れた値を異常値とみなす方法。異常値の発生頻度が低い場合に有効。
<br>
#### マハラノビス・タグチ法（MT法）
データの各次元間の相関を考慮した距離尺度を用いて、平均値から離れた値を異常値とみなす方法。次元数が大きい場合でも、次元削減を行うことなく異常検知を行うことができる。
<br>
#### ホテリングの$T^2$法
マハラノビス距離を拡張した手法で、データの平均値からのずれを検定統計量として用いて異常値を検出する方法。多変量データにも適用できる。
<br>
#### 密度比推定
正常データと異常データの確率密度比を推定することで異常を検知する手法。正常データと異常データの分布が大きく異なる場合に有効。また、正常データのみから異常検知モデルを構築することができるため、異常データが少ない場合でも適用することができる。
<br>

In [ ]:
from scipy.spatial.distance import mahalanobis
from scipy.stats import f
from sklearn.neighbors import KernelDensity


### 3σ法
# threshold: データの平均値から標準偏差の何倍離れたら異常値とみなすか
def three_sigma_method(data, threshold=3):
  mean = np.mean(data)
  std = np.std(data)
  lower_bound = mean - threshold * std
  upper_bound = mean + threshold * std
  if data.ndim == 1:
    anomaly_indices = np.where((data < lower_bound) | (data > upper_bound))[0]
  elif data.ndim >= 2:
    anomaly_indices = np.where(np.any((data < lower_bound) | (data > upper_bound), axis=1))[0]
  return data[anomaly_indices]

### マハラノビス距離
# マハラノビス距離: データの相関を考慮した距離尺度
# threshold_percentile: マハラノビス距離の分布におけるパーセンタイル値を超える
#                       距離を持つデータを異常値とみなす
def mahalanobis_distance_method(data, threshold_percentile=99):
  mean = np.mean(data, axis=0)
  if data.ndim == 1:
    std = np.std(data)

    # 標準偏差が0の場合、ゼロ除算を避けるために微小な値を加算
    if std == 0:
      std = 1e-6

    distances = np.abs((data - mean) / std)

    # 異常値とみなす閾値を設定
    threshold = np.percentile(distances, threshold_percentile)

    # 閾値を超えるデータを異常値として検出
    anomalies = data[distances > threshold]
  elif data.ndim >= 2:
    cov = np.cov(data, rowvar=False)
    inv_cov = np.linalg.inv(cov)

    distances = [mahalanobis(x, mean, inv_cov) for x in data]

    # 異常値とみなす閾値を設定
    threshold = np.percentile(distances, threshold_percentile)

    # 閾値を超えるデータを異常値として検出
    anomaly_indices = np.where(np.array(distances) > threshold)[0]
    anomalies = data[anomaly_indices]
  return anomalies

### ホテリングのT^2法
def hotelling_t2_method(data):
  mean = np.mean(data, axis=0)
  if data.ndim == 1:
    var = np.var(data)
    t2_values = [(x - mean)**2 / var for x in data]
    # f.ppf: F分布のパーセント点関数
    threshold = f.ppf(0.95, 1, len(data) - 1)  # 異常値とみなす閾値を設定
    anomalies = [data[i] for i, t2 in enumerate(t2_values) if t2 > threshold]
  elif data.ndim >= 2:
    mean = np.mean(data, axis=0)
    cov = np.cov(data, rowvar=False)
    inv_cov = np.linalg.inv(cov)
    t2_values = [mahalanobis(x, mean, inv_cov)**2 for x in data]
    threshold = f.ppf(0.95, data.shape[1], data.shape[0] - data.shape[1] - 1)  # 異常値とみなす閾値を設定
    anomaly_indices = np.where(np.array(t2_values) > threshold)[0]
    anomalies = data[anomaly_indices]
  return anomalies


# 正常データ生成
normal_data = np.random.normal(loc=25, scale=2, size=100)

# 異常データ生成
anomaly_data = np.random.normal(loc=25, scale=2, size=100)
# 異常値の追加
anomaly_indices = [10, 20, 30]  # 異常値のインデックス
anomaly_values = [35, 15, 32]  # 異常値
anomaly_data[anomaly_indices] = anomaly_values

# 結果の出力
anomalies = three_sigma_method(anomaly_data)
print("================== 3σ法 =================")
print("異常値:", anomalies)
print("\n")

anomalies = mahalanobis_distance_method(anomaly_data)
print("===== マハラノビス距離による異常検知 =====")
print("異常値:", anomalies)
print("\n")

anomalies = hotelling_t2_method(anomaly_data)
print("===== ホテリングのT^2法による異常検知 ====")
print("異常値:", anomalies)
print("\n")


================== 3σ法 =================
異常値: [35. 15.]


===== マハラノビス距離による異常検知 =====
異常値: [35.]


===== ホテリングのT^2法による異常検知 ====
異常値: [np.float64(35.0), np.float64(30.22775217743949), np.float64(15.0), np.float64(32.0), np.float64(19.028688325241813)]




## 非正規データの異常検知
### 例
ウェブサイトへのアクセス数の異常検知
<br>
<br>
### 手法

#### ガンマ分布の当てはめ
データがガンマ分布に従うと仮定し、その分布から大きく外れた値を異常値とみなす方法。データがガンマ分布に従う場合、高い精度で異常を検出できるが、従わない場合、精度が低下する。
<br>

#### カイ二乗分布への当てはめ
データがカイ二乗分布に従うと仮定し、その分布から大きく外れた値を異常値とみなす方法。主に、特徴量の値が正の値を取り、歪んだ分布をしている場合に有効。
<br>

#### $k$近傍法
各データポイントからk番目に近いデータポイントまでの距離を計算し、距離が大きいデータポイントを異常値と判定するアルゴリズム。$k$はデータセットのサイズの平方根よりも小さい奇数にすると良い。正常データと異常データの分布が明確に分かれていない場合でも、異常を検出できる。
<br>

#### $k$ means法
データを複数のクラスタに分割し、どのクラスタにも属さないデータポイントを異常値と判定するアルゴリズム。クラスタ数$k$を推定する手法としてエルボー法やシルエット分析などがある。
<br>

#### 混合ガウス分布モデル（Gaussian Mixture Model, GMM）
データが複数のガウス分布（正規分布）の混合で表現できると仮定し、低確率なデータ点を異常値とみなす方法。ガウス分布の数（コンポーネント数）を適切に設定する必要がある。
<br>

#### One-Class SVM
正常データのみを用いて、正常データの領域を学習するアルゴリズム。 学習した領域から外れたデータは異常値と判定する。正常データのみを用いて学習するため、異常データが少ない場合や入手が困難な場合でも適用できる。
<br>

#### [密度比推定](#scrollTo=1dE3Gampih0k&line=1&uniqifier=1)
<br>

#### 孤立フォレスト（Isolation Forest）
データポイントをランダムに分割していくことで、異常値を孤立させるアルゴリズム。異常値の割合が高いデータセットに対しても良好に機能する。
<br>

#### Local Outlier Factor (LOF)
データポイントの局所的な密度を計算し、密度が低いデータポイントを異常値と判定するアルゴリズム。クラスタの形状が複雑な場合でも、異常を検出できる。
<br>



In [ ]:
import scipy.stats as stats
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import IsolationForest


### ガンマ分布の当てはめ
def fitting_gamma_distribution(data):
  # データにガンマ分布を当てはめる
  shape, loc, scale = stats.gamma.fit(data)

  # 異常度を計算する
  anomaly_scores = 1 / stats.gamma.pdf(data, shape, loc, scale)

  # 閾値を設定する
  threshold = np.mean(anomaly_scores) + 3 * np.std(anomaly_scores)

  # 異常値を検出する
  anomalies = data[anomaly_scores > threshold]

  return anomalies


### k近傍法
def k_nearest_neighbors(data, k, threshold_percentile):
  data = data.reshape(-1, 1)

  # k近傍の計算
  knn = NearestNeighbors(n_neighbors=k)
  knn.fit(data)
  distances, indices = knn.kneighbors(data)

  # 異常度の算出
  anomaly_score = distances[:, -1]
  threshold = np.percentile(anomaly_score, threshold_percentile)

  # 異常値の判定
  anomaly_index = np.where(anomaly_score > threshold)

  # 異常値を検出する
  anomalies = data[anomaly_index]

  return anomalies


### 孤立フォレスト
def isolation_forest(data, threshold_percentile):
  data = data.reshape(-1, 1)

  # モデルの学習
  model = IsolationForest()
  model.fit(data)

  # 異常度の算出
  anomaly_score = model.decision_function(data)
  threshold = np.percentile(anomaly_score, threshold_percentile)

  # 異常値の判定
  anomaly_index = np.where(anomaly_score < threshold)

  # 異常値を検出する
  anomalies = data[anomaly_index]

  return anomalies


def power_law_distribution(alpha, xmin, xmax, size):
    """べき乗則に従う乱数を生成する関数

    Args:
        alpha (float): べき乗則の指数
        xmin (float): 最小値
        xmax (float): 最大値
        size (int): 生成する乱数の数

    Returns:
        numpy.ndarray: べき乗則に従う乱数
    """

    u = np.random.uniform(size=size)
    return (xmin ** (1 - alpha) + (xmax ** (1 - alpha) - xmin ** (1 - alpha)) * u) ** (1 / (1 - alpha))


# 正常データの生成
normal_data = power_law_distribution(alpha=2, xmin=1, xmax=100, size=1000)

# 異常データの生成
outliers = np.random.uniform(low=2*normal_data.max(), high=3*normal_data.max(), size=10)
anomaly_data = np.concatenate([normal_data, outliers])

all_data = np.concatenate([normal_data, anomaly_data])

#### ガンマ分布の当てはめ
anomalies = fitting_gamma_distribution(all_data)
print("============ ガンマ分布の当てはめ ==========")
print("異常値:", anomalies)
print("\n")

#### k近傍法
anomalies = k_nearest_neighbors(all_data, 5, 99.5)
print("================== k近傍法 =================")
print("異常値:", anomalies)
print("\n")

#### 孤立フォレスト (Isolation Forest)
anomalies = isolation_forest(all_data, 0.5)
print("=============== 孤立フォレスト ==============")
print("異常値:", anomalies)
print("\n")


============ ガンマ分布の当てはめ ==========
異常値: [204.32205607 235.47297516 230.19047005]


================== k近傍法 =================
異常値: [[ 79.62241035]
 [ 79.62241035]
 [184.92162139]
 [171.46768264]
 [196.0355937 ]
 [204.32205607]
 [194.53311106]
 [235.47297516]
 [167.19292325]
 [230.19047005]
 [171.22925385]]


=============== 孤立フォレスト ==============
異常値: [[184.92162139]
 [171.46768264]
 [179.44610057]
 [196.0355937 ]
 [204.32205607]
 [194.53311106]
 [235.47297516]
 [167.19292325]
 [230.19047005]
 [171.22925385]]




## 不要次元のある次元データの異常検知
### 例
顧客の購買データを用いた異常検知
<br>
<br>
### 手法
#### 主成分分析（PCA）
データの分散を最大化するように新しい軸を定義し、その軸にデータを射影することで次元を削減する手法。異常なデータ点は、主成分空間において、正常なデータ点から離れた位置に配置される傾向がある。データに非線形な関係がある場合、精度が低下する。
<br>

#### 確率的主成分分析（Probabilistic PCA）
PCAを確率モデルとして拡張した手法。データにノイズが含まれる場合に、よりロバストな結果が得られる。（＝データにノイズや外れ値が含まれていても、分析結果が大きく影響を受けにくい）データが正規分布に従わない場合、精度が低下する。
<br>

#### カーネル主成分分析（Kernel PCA）
非線形な関係を持つデータに対して、カーネル関数を使用して高次元空間に写像し、その空間でPCAを行うことで次元を削減する手法。
<br>

#### 因子分析（Factor Analysis）
観測変数の背後にある潜在変数を推定し、それらを用いて次元を削減する手法。観測変数間に相関が無い場合は精度が低下する。
<br>

#### 独立成分分析（ICA）
観測変数を、統計的に独立な成分に分解することで次元を削減する手法。観測されたデータが、いくつかの独立した信号源から混合されたものであると仮定し、それらの信号源を分離する。信号源の数が事前に分かっている必要がある。
<br>

#### t-distributed Stochastic Neighbor Embedding (t-SNE)
高次元データを低次元空間に埋め込む際に、データの局所的な構造を保持する様に設計された手法。異常検知は、t-SNEを用いて次元削減を行い、次元削減後の空間で異常検知を行う。
<br>

#### Uniform Manifold Approximation and Projection (UMAP)
高次元データを低次元空間に埋め込む際に、データのトポロジカルな構造を保持する様に設計された手法。異常検知は、t-SNEと同様に、次元削減後の空間で異常検知を行う。
<br>

#### [One-Class SVM](#scrollTo=FMOqmjNLiot5&line=33&uniqifier=1)
<br>

#### [孤立フォレスト（Isolation Forest）](#scrollTo=FMOqmjNLiot5&line=33&uniqifier=1)
<br>

#### [Local Outlier Factor (LOF)](#scrollTo=FMOqmjNLiot5&line=33&uniqifier=1)
<br>






In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV


### One-Class SVM
def one_class_svm(data, threshold_percentile):
  data = data.reshape(-1, 1)

  scaler = StandardScaler()
  scaled_data = scaler.fit_transform(data)

  # モデルの学習
  #param_grid = {'nu': [0.1, 0.2, 0.3], 'gamma': ['scale', 'auto']}
  model = OneClassSVM(nu=0.1, gamma='scale')
  model.fit(scaled_data)

  # 異常度の算出
  anomaly_score = model.decision_function(scaled_data)
  threshold = np.percentile(anomaly_score, threshold_percentile)

  # 異常値の判定
  anomaly_index = np.where(anomaly_score < threshold)

  # 異常値を検出する
  anomalies = data[anomaly_index]

  return anomalies


### Local Outlier Factor (LOF)
def local_outlier_factor(data):
  data = data.reshape(-1, 1)

  # モデルの学習
  model = LocalOutlierFactor()
  anomaly_score = model.fit_predict(data)

  # 異常値の判定
  anomaly_index = np.where(anomaly_score == -1)

  # 異常値を検出する
  anomalies = data[anomaly_index]

  return anomalies


#### 正常データの生成
# 顧客ID、年齢、性別、購入金額、購入頻度、不要な次元（ランダムな数値）を生成
num_customers = 100  # 顧客数
customer_ids = np.arange(1, num_customers + 1)
ages = np.random.randint(20, 60, size=num_customers)
genders = np.random.choice(['Male', 'Female'], size=num_customers)
purchase_amounts = np.random.randint(100, 10000, size=num_customers)
purchase_frequencies = np.random.randint(1, 10, size=num_customers)
unnecessary_dimension = np.random.rand(num_customers)

# データフレームを作成
normal_data = pd.DataFrame({
  'CustomerID': customer_ids,
  'Age': ages,
  'Gender': genders,
  'PurchaseAmount': purchase_amounts,
  'PurchaseFrequency': purchase_frequencies,
  'UnnecessaryDimension': unnecessary_dimension  # 不要な次元
})

#### 異常データの生成
anomaly_data = normal_data.copy()

# 異常値を挿入するインデックスをランダムに選択
# 例：顧客の10%を異常値とする
anomaly_indices = np.random.choice(anomaly_data.index, size=int(num_customers * 0.1), replace=False)

# 選択したインデックスのデータに異常値を代入
# 例：PurchaseAmountを極端に大きくする
anomaly_data.loc[anomaly_indices, 'PurchaseAmount'] = anomaly_data.loc[anomaly_indices, 'PurchaseAmount'] * np.random.uniform(100, 200, size=len(anomaly_indices)).astype(np.int64)

all_data = pd.concat([normal_data, anomaly_data], ignore_index=True)
numerical_data = all_data[['Age', 'PurchaseAmount', 'PurchaseFrequency', 'UnnecessaryDimension']]

#### One-Class SVM
anomalies = one_class_svm(numerical_data.values, 1)
print("=============== One-Class SVM ==============")
print("異常値:", anomalies)
print("\n")

# #### 孤立フォレスト（Isolation Forest）
anomalies = isolation_forest(numerical_data['PurchaseAmount'].values, 5)
print("=============== 孤立フォレスト ==============")
print("異常値:", anomalies)
print("\n")

# #### Local Outlier Factor (LOF)
anomalies = local_outlier_factor(numerical_data['PurchaseAmount'].values)
print("============ Local Outlier Factor ===========")
print("異常値:", anomalies)
print("\n")



=============== One-Class SVM ==============
異常値: [[ 764175.]
 [1588976.]
 [1609210.]
 [ 898144.]
 [ 411160.]
 [1432748.]
 [1382364.]
 [ 968022.]]


=============== 孤立フォレスト ==============
異常値: [[ 137966]
 [ 764175]
 [1588976]
 [1609210]
 [ 898144]
 [ 411160]
 [1432748]
 [1382364]
 [ 968022]
 [ 172834]]


============ Local Outlier Factor ===========
異常値: [[ 137966]
 [ 764175]
 [1588976]
 [1609210]
 [ 898144]
 [ 411160]
 [1432748]
 [1382364]
 [ 968022]
 [ 172834]]




## 入出力関係のあるデータの異常検知
### 例
製造工程における品質管理（入力データ：温度、圧力、処理時間など、出力データ：寸法、重量、強度など）
<br>

### 手法

#### 線形回帰モデル
正常データを用いて、入力と出力の関係を回帰モデルで学習し、新しいデータが入力された際に、回帰モデルの予測値と実際の出力値の差が大きい場合、異常と判定する。
<br>

#### リッジ回帰
線形回帰モデルの一種で、過学習を防ぐために正則化項を導入したもの。
<br>

#### ガウス過程回帰
非線形な回帰問題を解くための機械学習手法。観測データから入力と出力の関係を確率分布として学習する。この確率分布はガウス過程（任意の有限個の点における確率変数の集合が、常に多変量正規分布に従うような確率過程）で表現され、予測値だけでなく、予測値の不確実性も推定することができる。データの量が少ない場合でも、高い精度で予測できる。
<br>

#### 状態空間モデル
入出力関係を状態空間モデルで表現し、観測値と予測値の差を異常度として用いる方法。状態空間モデルは、観測できない状態変数を推定することができ、また、未来の値を予測するために用いることができる。
<br>

#### [One-Class SVM](#scrollTo=FMOqmjNLiot5&line=33&uniqifier=1)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge


### 線形回帰
def linear_regression(input_data, output_data):
  # 線形回帰モデルを学習
  model = LinearRegression()
  model.fit(input_data, output_data)

  # 予測値と実際の出力値の差を計算
  predicted_output = model.predict(input_data)
  residuals = output_data - predicted_output

  # 異常値を判定 (例：残差が大きい値を異常値とする)
  threshold = np.mean(residuals) + 2 * np.std(residuals)  # 閾値は調整が必要
  anomalies = input_data[np.abs(residuals) > threshold]

  return anomalies


### リッジ回帰
def ridge_regression(input_data, output_data):
  # リッジ回帰モデルを学習
  model = Ridge(alpha=1.0)  # alphaは正則化の強さを指定
  model.fit(input_data, output_data)

  # 予測値と実際の出力値の差を計算
  predicted_output = model.predict(input_data)
  residuals = output_data - predicted_output

  # 異常値を判定 (例：残差が大きい値を異常値とする)
  threshold = np.mean(residuals) + 2 * np.std(residuals)  # 閾値は調整が必要
  anomalies = input_data[np.abs(residuals) > threshold]

  return anomalies


def one_class_svm_io(input_data, output_data, threshold_percentile):
  # 入力データと出力データを結合
  data = np.concatenate([input_data, output_data.reshape(-1, 1)], axis=1)

  # データをスケーリング
  scaler = StandardScaler()
  scaled_data = scaler.fit_transform(data)

  # One-Class SVMモデルを学習
  model = OneClassSVM(nu=0.1, gamma='scale') # nu, gammaは必要に応じて調整
  model.fit(scaled_data)

  # 異常度を算出
  anomaly_score = model.decision_function(scaled_data)
  threshold = np.percentile(anomaly_score, threshold_percentile)

  # 異常値の判定
  anomaly_index = np.where(anomaly_score < threshold)

  # 異常があったinput_dataを抽出
  anomalies = input_data[anomaly_index]

  return anomalies


# 正常データの生成
num_samples = 100
input_data = np.random.rand(num_samples, 2) * 10  # 0-10の範囲の乱数を生成
output_data = 2 * input_data[:, 0] + 3 * input_data[:, 1] + np.random.randn(num_samples) * 2 # y = 2x1 + 3x2 + noise

# 異常データの生成
num_anomalies = 10
anomaly_input_data = np.random.rand(num_anomalies, 2) * 10 + 10  # 10-20の範囲の乱数を生成
anomaly_output_data = np.random.rand(num_anomalies) * 10  # 0-10の範囲の乱数を生成

# データの結合
input_data = np.concatenate([input_data, anomaly_input_data])
output_data = np.concatenate([output_data, anomaly_output_data])


#### 線形回帰
anomalies = linear_regression(input_data, output_data)
print("================== 線形回帰 ================")
print("異常値:", anomalies)
print("\n")

#### リッジ回帰
anomalies = ridge_regression(input_data, output_data)
print("================= リッジ回帰 ================")
print("異常値:", anomalies)
print("\n")

#### One-Class SVM
anomalies = one_class_svm_io(input_data, output_data, 1)
print("=============== One-Class SVM ==============")
print("異常値:", anomalies)
print("\n")

================== 線形回帰 ================
異常値: [[14.55048105 10.66020651]
 [16.59602256 15.54815574]
 [14.79007644 16.33790192]
 [10.62915535 13.87242548]]


================= リッジ回帰 ================
異常値: [[14.55048105 10.66020651]
 [16.59602256 15.54815574]
 [14.79007644 16.33790192]
 [10.62915535 13.87242548]]


=============== One-Class SVM ==============
異常値: [[ 1.9030021   0.0693305 ]
 [18.97792173 12.42478737]]




## 時系列データの異常検知
### 例
サーバーのCPU使用率の異常検知

### 手法

#### 自己回帰モデル
時系列データの過去の値を用いて、未来の値を予測するモデル。予測値と実測値の差が大きい場合に異常と判定する。データに自己相関、定常性（時系列データの平均、分散、自己相関などの統計的性質が時間とともに変化しないこと）がある場合に有効。
<br>

#### 移動平均法
一定期間のデータの平均値を計算し、その平均値から大きく外れた値を異常とみなす手法。データのトレンドを把握できる。
<br>

#### 指数平滑法
過去のデータに重み付けをして平均を計算することで、未来の値を予測する手法。直近のデータに大きな重みを置くことで、短期的な変動に対応することができる。
<br>

#### ARIMAモデル
時系列データの自己相関を考慮したモデルで、自己回帰(AR)、和分(I)、移動平均(MA) の3つの要素から構成される。AR、I、MAの要素を組み合わせることで、時系列データの様々なパターンを表現できる。

##### 自己回帰(AR)
過去のデータ値を予測に利用するプロセス。

##### 和分(I)
データの定常性を確保するために、データに差分を取るプロセス。

##### 移動平均(MA)
過去の予測誤差を予測に利用するプロセス。

<br>

#### [$k$近傍法](#scrollTo=FMOqmjNLiot5&line=7&uniqifier=1)
<br>

#### サポートベクターマシン (SVM)
正常データと異常データを分離する超平面を学習する手法。[One-Class SVM](#scrollTo=FMOqmjNLiot5&line=33&uniqifier=1)は正常データのみを用いる。高次元データや線形分離可能でない場合でも利用できる。
<br>

#### [孤立フォレスト (Isolation Forest)](#scrollTo=FMOqmjNLiot5&line=7&uniqifier=1)
<br>

#### LSTM (Long Short-Term Memory)
時系列データや自然言語処理などのシーケンシャルデータの分析に広く用いられる深層学習モデルでRNN (Recurrent Neural Network) の一種。過去の情報を記憶し、現在の出力に反映させることができる。

##### セル状態
過去の情報を記憶するためのユニット。時系列データの処理が進むにつれて、過去の情報が蓄積されていく。

##### ゲート機構
セル状態への情報の入力、出力、保持を制御する仕組み。現在の入力情報をセル状態にどれだけ反映させるかを制御する入力ゲート、セル状態の情報を現在の出力にどれだけ反映させるかを制御する出力ゲート、セル状態の情報をどれだけ保持するかを制御する忘却ゲートからなる。
<br>

#### 特異スペクトル変換法
時系列データの長期的な依存関係を学習できるニューラルネットワーク。正常な時系列データを学習し、学習データと大きく異なるパターンを持つデータを異常とみなす。
<br>

#### [状態空間モデル](#scrollTo=NWg4xZztixiP&line=23&uniqifier=1)

#### Autoencoder
データを低次元空間に圧縮し、復元するニューラルネットワーク。正常な時系列データを学習し、復元誤差が大きいデータを異常とみなす。
<br>

#### Variational Autoencoder (VAE)
Autoencoderを確率モデルとして拡張した手法。
<br>

#### Generative Adversarial Networks (GANs)
 正常な時系列データを生成する生成器と、生成されたデータが正常か異常かを判定する識別器を競合的に学習させる手法。識別器が正常と判定できないデータを異常とみなす。
 <br>

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


### 移動平均法
# window_size: 移動平均を計算する際に使用するデータの期間の長さ。
#              周期性がある場合は周期の長さを設定すると異常値を検出しやすくなる。
#              周期性がない場合は、変動が激しい時は小さく設定、変動が緩やかな時は
#              大きく設定すると異常値を検出しやすくなる。
#              （window_sizeを大きくすると短期的な変動の影響を受けにくくなる）
def moving_average_method(data, window_size):
    # 移動平均を計算
    moving_average = np.convolve(data, np.ones(window_size) / window_size, mode='same')

    # 異常値を判定
    threshold = 2 * np.std(data - moving_average)  # 閾値は調整が必要
    anomalies = data[np.abs(data - moving_average) > threshold]

    return anomalies


### ARIMAモデル
# order: (p, d, q)からなるタプル
#        p: 自己回帰(AR)モデルの次数。過去のデータポイントをいくつ参照するか。
#        d: 階差の次数。データの定常化（トレンドや季節性の除去）のために何回差分を取るか。
#        q: 移動平均(MA)モデルの次数。過去の予測誤差をいくつ参照するか。
def arima_model(data, order):
    # ARIMAモデルを学習
    model = ARIMA(data, order=order)
    model_fit = model.fit()

    # 予測値と実測値の差を計算
    residuals = model_fit.resid

    # 異常値を判定
    threshold = 2 * np.std(residuals)  # 閾値は調整が必要
    anomalies = data[np.abs(residuals) > threshold]

    return anomalies


### LSTM (Long Short-Term Memory)
# timesteps: 過去の何時点分のデータを使って予測を行うか
def lstm_model(data, timesteps):
    # データをLSTMモデルの入力形式に変換
    X = []
    y = []
    for i in range(timesteps, len(data)):
        X.append(data[i - timesteps:i])
        y.append(data[i])
    X = np.array(X)
    y = np.array(y)

    # LSTMモデルを構築
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(timesteps, 1)))
    model.add(LSTM(units=50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    # LSTMモデルを学習
    model.fit(X, y, epochs=100, batch_size=32)

    # 予測値と実測値の差を計算
    predictions = model.predict(X)
    residuals = y - predictions.flatten()

    # 異常値を判定 (例：残差が大きい値を異常値とする)
    threshold = 3 * np.std(residuals)  # 閾値は調整が必要

    # 異常値のインデックスを取得
    anomaly_indices = np.where(np.abs(residuals) > threshold)[0]

    # 元のデータのサイズに合うようにインデックスを調整
    anomaly_indices = anomaly_indices + timesteps

    # 異常値を検出する
    anomalies = data[anomaly_indices]

    return anomalies


# CPU使用率データの例
cpu_usage = np.array([20, 22, 25, 23, 21, 24, 26, 28, 80, 25])  # 80 が異常値


#### 移動平均法
anomalies = moving_average_method(cpu_usage, window_size=3)
print("================== 移動平均法 ================")
print("異常値:", anomalies)
print("\n")

#### ARIMAモデル
anomalies = arima_model(cpu_usage, order=(3, 0, 0))  # orderはモデルの次数を指定
print("================= ARIMAモデル ================")
print("異常値:", anomalies)
print("\n")

#### LSTM (Long Short-Term Memory)
anomalies = lstm_model(cpu_usage, timesteps=3)  # timestepsは過去のデータを使用する期間を指定
print("==================== LSTM ====================")
print("異常値:", anomalies)
print("\n")


================== 移動平均法 ================
異常値: [80]


================= ARIMAモデル ================
異常値: [80]




/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 1442.6117
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - loss: 1434.9305
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 1427.4053
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 1420.0138
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 1412.7184
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 1405.4832
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - loss: 1398.2736
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 1391.0521
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 1383.7780
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 1376.4049
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - loss: 1368.8828
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - loss: 1361.1581
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - loss: 1353.1764
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 1344.8865
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━

## 変数間に関係があるデータの異常検知
### 例
クレジットカードの不正利用検知（深夜に高額な取引が行われた場合、不正利用の可能性が高い、特定の場所で頻繁に取引が行われた場合、盗難カードの可能性が高いなど）

### 手法
#### 疎構造学習
高次元データにおいて、変数間の関係が疎であること（多くの要素が0であること）を仮定して、データの構造を学習する手法。正常なデータにおける変数間の関係を疎構造として学習し、その構造から外れるデータを異常とみなす。
<br>

#### ベイジアンネットワーク
確率変数間の因果関係を表現したグラフィカルモデル。変数間の条件付き確率を学習し、異常な条件付き確率を持つデータを異常とみなす。多くの変数と因果関係を持つ複雑なシステムを表現するのに適している。
<br>

#### マルコフ確率場（Markov Random Field, MRF）
確率変数間の相互作用を表現したグラフィカルモデル。正常データのみを用いてMRFモデルを学習し、学習済みのMRFモデルに基づいて異常度を算出することで異常を検出する。異常度は、データがMRFモデルで表現される確率分布からどれだけ逸脱しているかを表す。
<br>

#### [k-means法](#scrollTo=FMOqmjNLiot5&line=7&uniqifier=1)
<br>

#### DBSCAN（Density-Based Spatial Clustering of Applications with Noise）
密度ベースのクラスタリングアルゴリズム。データポイントを、高密度領域にあるデータポイントと低密度領域にあるデータポイント（ノイズ）に分類し、低密度領域にあるデータポイントを異常とみなす。密度ベースでクラスタリングを行うため、クラスタの形状が複雑な場合でも、適切にクラスタリングすることができる。また、クラスタの数を事前に指定する必要はない。
<br>

#### [One-Class SVM](#scrollTo=FMOqmjNLiot5&line=33&uniqifier=1)
<br>

#### [孤立フォレスト (Isolation Forest)](#scrollTo=FMOqmjNLiot5&line=7&uniqifier=1)
<br>

#### [Local Outlier Factor (LOF)](#scrollTo=FMOqmjNLiot5&line=33&uniqifier=1)
<br>

In [ ]:
from sklearn.preprocessing import RobustScaler


#### One-Class SVM
def one_class_svm_relationship(data, threshold_percentile):
  anomalies_per_column = pd.DataFrame()

  for column in data.columns:
    scaler = RobustScaler()
    scaled_data = scaler.fit_transform(data[[column]])

    # モデルの学習
    # nu:     学習データの中で異常値と判定されるデータの割合の上限
    #         nuを大きくすると、境界が緩やかになり、正常データを異常値と誤判定する
    #         可能性は低くなるが、異常値を検出できない可能性も高まる
    # kernel: モデルがデータの非線形性をどのように扱うか
    # gamma:  カーネル関数の影響範囲
    #         gammaの値が大きいと境界がより複雑で、データポイントに強くフィットする
    #         ようになるが過学習のリスクが高まる
    model = OneClassSVM(nu=0.3, kernel='rbf', gamma='scale')
    model.fit(scaled_data)

    # 異常度の算出
    anomaly_score = model.decision_function(scaled_data)
    threshold = np.percentile(anomaly_score, threshold_percentile)

    # 異常値の判定
    anomaly_index = np.where(anomaly_score < threshold)[0]

    # 異常値を検出する
    anomalies_per_column[column] = data.iloc[anomaly_index][column]

  return anomalies_per_column


#### 孤立フォレスト (Isolation Forest)
def isolation_forest_relationship(data, threshold_percentile):
  anomalies_per_column = pd.DataFrame()

  for column in data.columns:
    # Isolation Forestモデルを学習
    model = IsolationForest()
    model.fit(data[[column]])

    # 異常度を算出
    anomaly_score = model.decision_function(data[[column]])
    threshold = np.percentile(anomaly_score, threshold_percentile)

    # 異常値の判定
    anomaly_index = np.where(anomaly_score < threshold)[0]

    # 異常値を検出する
    anomalies_per_column[column] = data.iloc[anomaly_index][column]

  return anomalies_per_column


#### Local Outlier Factor (LOF)
def local_outlier_factor_relationship(data):
  # Local Outlier Factorモデルを学習
  # n_neighbors: 近傍数
  #              近傍数が少ない場合、局所的な外れ値に敏感になり、小さなクラスタも外れ値として
  #              検出される可能性がある
  model = LocalOutlierFactor(n_neighbors=2)  # n_neighborsはkの値を指定
  anomaly_scores = model.fit_predict(data)

  # 異常値を判定 (例：異常度が-1の値を不正利用と判定)
  # fit_predictは以下を返す
  # 1: 正常なデータポイント
  # -1: 異常なデータポイント (外れ値)
  anomalies = transaction_data.iloc[[i for i, score in enumerate(anomaly_scores) if score == -1]]

  return anomalies


# 取引データの例
transaction_data = pd.DataFrame({
    'amount':   [100, 120, 130, 110, 125, 115, 135, 105, 10000, 120],
    'time':     [ 10,  12,  13,  11,  12,  11,  13,  10,   20,  12],
    'location': [  1,   2,   1,   3,   2,   1,   3,   2,    5,   1]
})

#### One-Class SVM
anomalies = one_class_svm_relationship(transaction_data, 10)
print("================ One-Class SVM ===============")
print("異常値:", anomalies)
print("\n")

### 孤立フォレスト
anomalies = isolation_forest_relationship(transaction_data, 10)
print("=============== 孤立フォレスト ===============")
print("異常値:", anomalies)
print("\n")

#### Local Outlier Factor
anomalies = local_outlier_factor_relationship(transaction_data)
print("============ Local Outlier Factor ============")
print("異常値:", anomalies)
print("\n")


================ One-Class SVM ===============
異常値:    amount  time  location
8   10000    20         5


=============== 孤立フォレスト ===============
異常値:    amount  time  location
8   10000    20         5


============ Local Outlier Factor ============
異常値:    amount  time  location
8   10000    20         5


